In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Move to the project folder
%cd /content/drive/MyDrive/perso/yolov9_driver_license
!ls

/content/drive/MyDrive/perso/yolov9_driver_license
ecommerce_mongodb_vector_search.ipynb  predicted_images  split_data		      yolov8n.pt
faces_vector_search		       prediction.ipynb  train.ipynb		      yolov9c.pt
pred				       runs		 vision-api-credentials.json


In [ ]:
# Libraries importation
from ultralytics import YOLO
import cv2
import os
from glob import glob
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np
from google.cloud import vision
import os
from glob import glob
#from PIL import Image
import io
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/perso/yolov9_driver_license/vision-api-credentials.json"
client = vision.ImageAnnotatorClient()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Load the trained model
image = '/content/drive/MyDrive/perso/yolov9_driver_license/split_data/test/images/midv2020-alb-id_18_jpg.rf.7e639a73a51ad0e47659ee6afd8a81f3.jpg'
trained_model = YOLO('/content/drive/MyDrive/perso/yolov9_driver_license/runs/detect/train3/weights/best.pt')
# Run inference with the YOLOv9c model on the 'bus.jpg' image
results = trained_model(image)

for r in results:
  for c in r.boxes: #.cls:
    #print(trained_model.names[int(c.cls)])
    #print(c.xyxy.tolist())
    for i in c.xyxy.tolist():
      x1 = int(i[0])
      y1 = int(i[1])
      x2 = int(i[2])
      y2 = int(i[3])
      #print(trained_model.names[int(c.cls)], [x1,y1,x2,y2])
      img = cv2.imread(image)
      roi = img[y1:y2, x1:x2]
      name = f"/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/{trained_model.names[int(c.cls)]}.jpg"
      cv2.imwrite(name, roi)
results[0].save(filename='/content/drive/MyDrive/perso/yolov9_driver_license/pred/result.jpg')


image 1/1 /content/drive/MyDrive/perso/yolov9_driver_license/split_data/test/images/midv2020-alb-id_18_jpg.rf.7e639a73a51ad0e47659ee6afd8a81f3.jpg: 416x640 1 birth_date, 1 birth_place, 1 document_number, 1 expiry_date, 1 face_photo, 1 gender, 1 issue_authority, 1 issue_date, 1 name, 1 nationality, 1 personal_number, 1 signature, 1 surname, 2229.1ms
Speed: 19.1ms preprocess, 2229.1ms inference, 32.9ms postprocess per image at shape (1, 3, 416, 640)


'/content/drive/MyDrive/perso/yolov9_driver_license/pred/result.jpg'

In [ ]:
for i in glob("/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/*"):
  print(i)
  with io.open(i, 'rb') as image_file:
    content = image_file.read()

  #image = vision.types.Image(content=content)   #vision.Image(content=content)
  image = vision.Image(content=content)


  response = client.text_detection(image=image)
  texts = response.text_annotations
  ocr_result = []
  for text in texts:
      ocr_result.append(text.description)
      #print(text.description)
  ocr_result = list(set(ocr_result))
  ocr_result

/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/face_photo.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/issue_date.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/surname.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/expiry_date.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/issue_authority.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/signature.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/birth_date.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/personal_number.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/nationality.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/document_number.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/name.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/

In [ ]:
import io
import os
# Import the glob function to search for files based on a pattern
from glob import glob

# Imports the Google Cloud client library
from google.cloud import vision

# Instantiates a client
client = vision.ImageAnnotatorClient()
ocr_result = []

# The name of the image file to annotate
# file_name = os.path.abspath('/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/birth_date.jpg')
for file_name in glob("/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/*"):
  file_name_key = file_name.split("/")[-1].split(".jpg")[0]
  if file_name_key not in ["signature", "face_photo"]:
    # Loads the image into memory
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # Performs label detection on the image file
    response = client.text_detection(image=image)


    for el in response.text_annotations:
        #print(el.description)
        ocr_result.append((file_name_key,el.description))
ocr_result = list(set(ocr_result))
ocr_result


[('birth_date', '15-05-1972'),
 ('personal_number', 'H25515833M'),
 ('birth_place', 'Elbasan'),
 ('birth_place', 'Gjinar'),
 ('nationality', '/'),
 ('expiry_date', '24-03-2027'),
 ('issue_date', '24-03-2017'),
 ('nationality', 'Shqiptare/Albanian'),
 ('nationality', 'Shqiptare'),
 ('issue_authority', 'MPB'),
 ('birth_place', ','),
 ('document_number', '488484539'),
 ('name', 'Altea'),
 ('surname', 'Bardici'),
 ('nationality', 'Albanian'),
 ('birth_place', 'Gjinar,Elbasan')]

In [ ]:
grouped_dict = {}

for category, item in ocr_result:
    if category not in grouped_dict:
        grouped_dict[category] = []
    grouped_dict[category].append(item)

grouped_dict

{'birth_date': ['15-05-1972'],
 'personal_number': ['H25515833M'],
 'birth_place': ['Elbasan', 'Gjinar', ',', 'Gjinar,Elbasan'],
 'nationality': ['/', 'Shqiptare/Albanian', 'Shqiptare', 'Albanian'],
 'expiry_date': ['24-03-2027'],
 'issue_date': ['24-03-2017'],
 'issue_authority': ['MPB'],
 'document_number': ['488484539'],
 'name': ['Altea'],
 'surname': ['Bardici']}

In [ ]:
for key, value_list in grouped_dict.items():
    # Find the longest element in the list
    longest_element = max(value_list, key=len)
    # Update the dictionary to keep only the longest element for this key
    grouped_dict[key] = [longest_element]

grouped_dict

{'birth_date': ['15-05-1972'],
 'personal_number': ['H25515833M'],
 'birth_place': ['Gjinar,Elbasan'],
 'nationality': ['Shqiptare/Albanian'],
 'expiry_date': ['24-03-2027'],
 'issue_date': ['24-03-2017'],
 'issue_authority': ['MPB'],
 'document_number': ['488484539'],
 'name': ['Altea'],
 'surname': ['Bardici']}

# **TRANSFER DATA TO MONGODB CLOUD**

In [ ]:
# Loading the Model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = ViTModel.from_pretrained("google/vit-base-patch16-224")

def generate_image_embeddings(myImage):
    """
        This function takes as input an image and returns its embeddings generated via a transfromer based network.

        Params:
            Inputs:
                image : image path, str
            Outputs:
                embeddings : list
    """
    image = Image.open(myImage).convert('RGB')
    inputs = feature_extractor(images=image, return_tensors="pt")

    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.tolist()
    return embeddings[0][0]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import base64
from PIL import Image
from io import BytesIO

image_to_process = "/content/drive/MyDrive/perso/yolov9_driver_license/pred/cropping_img/face_photo.jpg"
# encode image into base64
with open(image_to_process, "rb") as image_file:
    image_to_base64 = base64.b64encode(image_file.read())

In [ ]:
image_to_base64

b'...'

In [ ]:
# create a list to store images and their corresponding embeddings. They are pair as dictionnary
all_data = list()
dict_image_processing = {"image_base64": image_to_base64,
                         "image_embeddings": generate_image_embeddings(image_to_process)}
print("Done")

Done


In [ ]:
def Merge(dict1, dict2):
    return(dict2.update(dict1))

# This returns None
print(Merge(dict_image_processing, grouped_dict))

# changes made
print(grouped_dict)

None
{'birth_date': ['15-05-1972'], 'personal_number': ['H25515833M'], 'birth_place': ['Gjinar,Elbasan'], 'nationality': ['Shqiptare/Albanian'], 'expiry_date': ['24-03-2027'], 'issue_date': ['24-03-2017'], 'issue_authority': ['MPB'], 'document_number': ['488484539'], 'name': ['Altea'], 'surname': ['Bardici'], 'image_base64': b'...', 'image_embeddings': [1.1406742334365845, -1.1019872426986694, 0.02654486894607544, -0.4365456998348236, -0.807961106300354, 1.2307156324386597, -0.4695498049259186, 1.493324637413025, 0.6318630576133728, -2.1431970596313477, 0.29421332478523254, -1.8914904594421387, 1.0647979974746704, 1.9797441959381104, -0.7640013694763184, 1.456388235092163, -1.1315423250198364, 0.22799424827098846, -1.2785534858703613, -0.4273073375225067, -0.8621115684509277, -0.9676133394241333, 0.9226253628730774, 0.5855967402458191, -0.7823998332023621, -0.13277310132980347, 0.3123549520969391, -0.9412374496459961, 0.4821898639202118, -1.072659969329834, 1.4025874137878418, -0.31670

In [ ]:
# Import libraries

import os
from dotenv import load_dotenv
from pymongo import MongoClient

# Load .env file that contain your Atlas cluster credentials
load_dotenv()

MONGODB_URI = "enter your MongoDB Atlas url string"

# Connect to MongoDB cluster with MongoClient
client = MongoClient(MONGODB_URI)

# Get reference to your database
db = client.id_documents_DB

# Get reference to your collection
ID_DB_collection = db.id_collections

# Expression that inserts the images+embeddings documents into the collection.
result = ID_DB_collection.insert_one(grouped_dict)
#result = luna_label_DB_collection.insert_one(new_info)

document_id = result.inserted_id
print(f"_id of inserted document: {document_id}")
print("Transfer Done!")

client.close()

_id of inserted document: 65f98b0a0f43240d1e998ce0
Transfer Done!


In [ ]:
import numpy as np

from numpy.linalg import norm
#from sentence_transformers import SentenceTransformer
from glob import glob

In [ ]:
def generate_image_embeddings(myImage):
    """
        This function takes as input an image and returns its embeddings generated via a transfromer based network.

        Params:
            Inputs:
                image : image path, str
            Outputs:
                embeddings : list
    """
    image = Image.open(myImage).convert('RGB')
    inputs = feature_extractor(images=image, return_tensors="pt")

    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.tolist()
    return embeddings[0][0]


images_embeddings = list()
image_list_names = list()
for i in glob("/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/*"):
  image_list_names.append(i)
  load_img = generate_image_embeddings(i)
  images_embeddings.append(load_img)

res = dict(zip(image_list_names, images_embeddings))

# vector embeddings created from dataset
embeddings = images_embeddings

# query vector embedding
query_embedding = generate_image_embeddings("/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/barack_obama_1.jpg")

# define our distance metric
def cosine_similarity(a, b):
    return np.dot(a, b)/(norm(a)*norm(b))

# run semantic similarity search
print("Query Result:")
myresult = list()
for e, s in zip(embeddings, images_embeddings):
  print(s, " -> similarity score = ",cosine_similarity(e, query_embedding))
  myresult.append(s)

Query Result:
[0.052251338958740234, -0.6797881722450256, -1.69539475440979, 0.3281848728656769, -0.4613405466079712, -0.03185947239398956, -0.17588646709918976, 0.34543463587760925, 0.838137149810791, -0.2199128419160843, 2.0933756828308105, -1.5729217529296875, 0.8106184005737305, 0.17796698212623596, -0.5061081647872925, -1.9873825311660767, 0.3318694531917572, -0.11428001523017883, -0.7409568428993225, 0.8861361742019653, -0.9250240921974182, 0.07967691123485565, 0.8173918724060059, 0.64329594373703, -1.7140175104141235, -0.5536806583404541, 1.6650012731552124, -1.1708139181137085, -0.23530837893486023, -0.42529240250587463, -0.2849864661693573, -1.9548165798187256, -0.2725015878677368, 0.14101029932498932, 0.6520827412605286, 0.44193464517593384, 0.28832313418388367, -0.20670627057552338, 0.06719363480806351, 0.39637473225593567, 1.2772587537765503, 0.3488902747631073, -0.7777323126792908, 0.8966547250747681, 0.4087558090686798, 0.33977964520454407, -0.2999972403049469, 0.66384649

In [ ]:
print(myresult)

[[0.052251338958740234, -0.6797881722450256, -1.69539475440979, 0.3281848728656769, -0.4613405466079712, -0.03185947239398956, -0.17588646709918976, 0.34543463587760925, 0.838137149810791, -0.2199128419160843, 2.0933756828308105, -1.5729217529296875, 0.8106184005737305, 0.17796698212623596, -0.5061081647872925, -1.9873825311660767, 0.3318694531917572, -0.11428001523017883, -0.7409568428993225, 0.8861361742019653, -0.9250240921974182, 0.07967691123485565, 0.8173918724060059, 0.64329594373703, -1.7140175104141235, -0.5536806583404541, 1.6650012731552124, -1.1708139181137085, -0.23530837893486023, -0.42529240250587463, -0.2849864661693573, -1.9548165798187256, -0.2725015878677368, 0.14101029932498932, 0.6520827412605286, 0.44193464517593384, 0.28832313418388367, -0.20670627057552338, 0.06719363480806351, 0.39637473225593567, 1.2772587537765503, 0.3488902747631073, -0.7777323126792908, 0.8966547250747681, 0.4087558090686798, 0.33977964520454407, -0.2999972403049469, 0.663846492767334, 0.90

In [ ]:
# creating a new dictionary
my_dict = res #{"java":100, "python":112, "c":11}

# list out keys and values separately
key_list = list(my_dict.keys())
val_list = list(my_dict.values())

# print key with val 100
for t in myresult:
  position = val_list.index(t)
  print(key_list[position])

/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/barack_obama_2.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/barack_obama_3.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/barack_obama_1.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/joe_biden_2.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/elon_musk_1.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/joe_biden_1.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/elon_musk_4.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/elon_musk_2.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/barack_obama_4.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/elon_musk_3.jpg
/content/drive/MyDrive/perso/yolov9_driver_license/faces_vector_search/joe_biden_3.jpg
